In [67]:
!pip install agno dotenv ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 45.3 MB/s eta 0:00:00


In [69]:
from agno.agent import Agent
from agno.team import Team
from ddgs import DDGS
import os

In [76]:
os.environ["OPENAI_API_KEY"] = "-_-"
os.environ["OPENAI_BASE_URL"] = "https://openrouter.ai/api/v1"

In [87]:
def flow_search(zapros):
    ddg = DDGS()
    rezultaty = list(ddg.text(zapros, max_results=5))
    for i, rez in enumerate(rezultaty, 1):
        print(rez['title'])
        print(rez['href'])
        print(rez['body'][:100])
        print(' '*21)
    return rezultaty

flow_results = flow_search("найди рецепт лазаньи")

Лазанья , рецепты на RussianFood.com: 174 рецепта лазаньи
https://www.RussianFood.com/recipes/bytype/?fid=659
Лазанья (классический рецепт ). Этот рецепт лазаньи близок к классическому - с томатно-мясным соусом
                     
Простая лазанья с фаршем в духовке рецепт фото... - 1000.menu
https://1000.menu/cooking/44000-lazanya-klassicheskaya-s-farshem-v-duxovke-prostaya
Фарш мясной — 800 гр, помидоры — 2 шт., лук — 2 шт., листы лазаньи — 10 шт., чеснок — 2 зубч., томат
                     
Лазанья - рецепты с фото. Как приготовить лазанью дома?
https://www.iamcook.ru/showsubsection/lasagna
Подборка рецептов приготовления вкусной лазаньи шаг за шагом с фото.
                     
Как приготовить лазанью с фаршем: пошаговый рецепт
https://www.nur.kz/food/recipes/1746642-lazana-s-farsem-recept-posagovo/
8. Припустите лазанью в кипятке. Лазанья припущена в кипятке. Листки лазаньи припускают в кипятке: Y
                     
Лазанья Болоньезе | Рецепты Barilla
https://barilla.ru/rec

In [89]:
def search_tool(zapros: str):
    ddg = DDGS()
    rezultaty = list(ddg.text(zapros, max_results=3))
    output = ""
    for i, r in enumerate(rezultaty, 1):
        output += f"{r['title']}\n{r['href']}\n\n"
    return output

def agent_search(zapros: str):
    agent = Agent(
        name="Searcher",
        tools=[search_tool],
        instructions="Ты агент поиска. Используй search_tool для поиска информации в интернете. Предоставляй точные и проверенные сведения.",
        markdown=False,
        model="openrouter:xiaomi/mimo-v2-flash:free",
    )

    otvet = agent.run(zapros)
    print("Результат поиска агента:")
    print(otvet.content)
    return otvet.content

agent_results = agent_search("music 'madk1d'")

Результат поиска агента:
Сначала я должен понять, что пользователь имел в виду, написав "music 'madk1d'". Возможно, это название песни, альбома, исполнителя или что-то связанное с музыкой. "madk1d" может быть именем исполнителя, названием трека или альбома. Также возможно, что это опечатка или специфический запрос.

Поскольку у меня есть инструмент поиска, я могу выполнить поиск по запросу "madk1d music" или что-то подобное, чтобы выяснить, что это такое.

Давайте начнем с выполнения поиска.По результатам поиска, кажется, что "madk1d" может быть связан с Mack 10, американским рэпером. Также есть упоминание о Martine Rose и какой-то песне. Но нужно уточнить.

Возможно, пользователь имел в виду Mack 10, но написал "madk1d" по ошибке. Или это может быть что-то другое.

Давайте выполним еще один поиск, чтобы уточнить.Теперь я вижу, что есть артист с именем "madk1d" на YouTube Music и Genius. Похоже, это действительно музыкальный исполнитель.

Давайте посмотрим на страницу Genius для madk1d

In [92]:
def search_web(zapros: str):
    ddg = DDGS()
    rezultaty = list(ddg.text(zapros, max_results=10))
    output = ""
    for r in rezultaty:
        output += f"{r['title']}{r['href']}{r['body'][:100]}"
    return output

def team_search(zapros):
    searcher = Agent(
        name="Searcher",
        role="Search the web for information",
        tools=[search_web],
        markdown=False,
        model="openrouter:xiaomi/mimo-v2-flash:free"
    )
    analyzer = Agent(
        name="Analyzer",
        role="Analyze search results and extract key information",
        markdown=False,
        model="openrouter:xiaomi/mimo-v2-flash:free"
    )
    team = Team(
        members=[searcher, analyzer],
        name="Search Team",
        model="openrouter:xiaomi/mimo-v2-flash:free"
    )
    otvet = team.run(zapros)
    print(otvet.content)
    return otvet

team_results = team_search("кто такой парламентер")

Давайте я помогу вам разобраться, что такое парламентер.

Для этого мне нужно найти актуальную информацию об этом термине. Я обращусь к специалисту по поиску информации.Анализирую содержимое страницы Википедии о парламентёре. В статье подробно раскрывается значение термина:
- Парламентёр — лицо, уполномоченное вступать в переговоры с противником.
- История использования: ведение переговоров о перемирии, сдаче в плен, обмене пленными.
- Статус: имеет неприкосновенность, но может быть задержан, если нарушает условия переговоров.
- Современное применение: используется в вооруженных конфликтах и миротворческих операциях.
- Происхождение термина: от французского "parlementaire", что связано с парламентом.

Теперь посмотрим другие результаты для дополнительной информации. Открываем словарь Даля.
